In [76]:
import osmnx as ox
import pandas as pd
import geopandas as gpd


In [77]:
columns_to_keep = ['geometry', 'name', 'id', 'description', 'area_type', 'category']

## Adapt the boundaries and file name to the desired environment to retrieve data from

Make sure you created a folder

In [78]:
# boundaries = [
#     'Alphen aan den Rijn, Zuid-Holland, Netherlands',
#     'Waddinxveen, Zuid-Holland, Netherlands',
#     'Boskoop, Zuid-Holland, Netherlands'
# ]

# boundaries = ['Breda, Noord-Brabant, Netherlands']

boundaries = ['Borsele, Zeeland, Netherlands']

In [79]:
# file_name = 'alphen-waddinxveen.geojson'
# file_name = 'breda.geojson'
file_name = 'borsele.geojson'

In [80]:
# folder_name = 'alphen-waddinxveen'
# folder_name = 'breda'
folder_name = 'borsele'

# Mapping

In [ ]:
def determine_area_type_from_row(row):
    # Transportation
    if "highway" in row:
        if row["highway"] in ["motorway", "trunk", "primary"]:
            return "Motorways and major roads"
        elif row["highway"] in ["secondary", "tertiary", "residential"]:
            return "Regional roads"
        elif row["highway"] in ["track", "unclassified", "service"]:
            return "Tracks and rural access roads"
        elif row["highway"] in ["footway", "cycleway", "path", "bridleway" ,"pedestrian"]:
            return "Pedestrian and cycling paths"
        elif row["highway"] in ["living_street"]:
            return "Living and residential streets"

    if "railway" in row and row["railway"] in ["rail", "light_rail"]:
        return "Railways"

    if "bridge" in row and str(row["bridge"]).lower() == "yes":
        return "Bridges"

    # Power-related
    if "power" in row and row["power"] in ["line", "tower"]:
        return "Power lines"
    if "power" in row and row["power"] == "plant":
        return "Power plants"

    # Man-made infrastructure
    if "man_made" in row:
        val = row["man_made"]
        if val == "waste_water_plant":
            return "Power plants"
        elif val in ["communications_tower", "mast"]:
            return "Communication towers"
        elif val in ["windmill", "wind_turbine"]:
            return "Wind turbines"
        elif val in ["water_tower", "silo", "chimney", "tank"]:
            return "High infrastructures"

    # Land use types
    if "landuse" in row:
        val = row["landuse"]
        if val == "industrial":
            return "Industrial zones"
        elif val == "commercial":
            return "Commercial zones"
        elif val == "retail":
            return "Retail zones"
        elif val == "residential":
            return "Residential areas"
        elif val == "recreation_ground":
            return "Recreational zones"
        elif val in ["farmland", "orchard"]:
            return "Agricultural lands"

    # Nature areas
    if "landuse" in row:
        val = row["landuse"]
        if val in ["forest", "wood"]:
            return "Forests and woodlands"
        elif val in ["meadow", "scrub", "grassland", "grass"]:
            return "Meadows and open grass"

    if "natural" in row and row["natural"] == "wetland":
        return "Wetlands"

    if "wetland" in row and row["wetland"] in ["marsh", "bog", "fen"]:
        return "Wetlands"

    # Water areas
    if "waterway" in row and row["waterway"] in ["river", "stream", "canal"]:
        return "Rivers, canals and streams"

    if "water" in row and row["water"] in ["lake", "reservoir", "pond"]:
        return "Lakes and ponds"

    if "water" in row and row["water"] in ["reservoir"]:
        return "Reservoirs"

    # Public facilities
    if "amenity" in row:
        if row["amenity"] in ["school", "kindergarten", "university"]:
            return "Schools and universities"
        if row["amenity"] == "hospital":
            return "Hospitals"
        if row["amenity"] == "nursing_home":
            return "Care homes"
        if row["amenity"] == "prison":
            return "Prisons"
        if row["amenity"] == "place_of_worship":
            return "Religious sites"
        if "historic" in row:
            return "Cultural sites"
        if "tourism" in row and row["tourism"] == "attraction":
            return "Cultural sites"
        if "landuse" in row and row["landuse"] == "cemetery":
            return "Cemeteries"
        if "leisure" in row and row["leisure"] == "park":
            return "Parks"

    # Airports and aviation
    if "aeroway" in row:
        val = row["aeroway"]
        if val in ["aerodrome", "airstrip", "heliport", 'apron']:
            return "Airports and airfields"
        elif val == "helipad":
            return "Helipads"
        elif val in ["runway", "aprons"]:
            return "Runways and aprons"
        elif val in ["terminal", "gate"]:
            return "Passenger terminals"
        elif val in ["hangar"]:
            return "Hangars and support buildings"

    if "man_made" in row and row["man_made"] == "control_tower":
        return "Hangars and support buildings"


    return "Unknown"



## Transportation infrastructure

In [7]:
transportation_tags = {
    "highway": [
        "motorway", "trunk", "primary",         # Motorways and major roads
        "secondary", "tertiary", "residential", # Regional/local roads
        "living_street",                        # Living streets    
        "track", "unclassified", "service",     # Tracks and rural access roads
        "footway", "cycleway", "path", "bridleway", "pedestrian"  # Pedestrian/cycling paths
    ],
    "railway": [
        "rail", "light_rail"                    # Railways
    ],
    "bridge": [
        "yes"                                   # Bridges and elevated corridors
    ]
}

In [8]:
# Get data from OSM
transportation_data = ox.features_from_place(boundaries, tags=transportation_tags)

In [9]:
transportation_data.reset_index(inplace=True)

In [10]:
transportation_data["area_type"] = transportation_data.apply(determine_area_type_from_row, axis=1)


In [11]:
transportation_data['category'] = "Transportation"

In [12]:
for column in columns_to_keep:
    if column not in transportation_data.columns:
        transportation_data[column] = None

transportation_data = transportation_data[columns_to_keep]

In [13]:
transportation_data['area_type'].unique()

array(['Pedestrian and cycling paths', 'Tracks and rural access roads',
       'Regional roads', 'Living and residential streets',
       'Motorways and major roads', 'Bridges', 'Railways'], dtype=object)

In [14]:
transportation_data.to_file(f'/Users/cmartens/Documents/Thesis/model/OSM_data/output/{folder_name}/transportation_{file_name}')

## Energy/Industrial

In [15]:
energy_industrial_tags = {
    "power": ["line", "plant", "tower"],
    "man_made": [
        "waste_water_plant", 
        "communications_tower", 
        "mast", 
        "windmill", 
        "wind_turbine", 
        "water_tower", 
        "silo", 
        "chimney", 
        "tank"
    ]
}

In [16]:
industrial_data = ox.features_from_place(boundaries, tags=energy_industrial_tags)

In [17]:
industrial_data.reset_index(inplace=True)

In [18]:
industrial_data['category'] = "Energy/Industrial"

In [19]:
industrial_data["area_type"] = industrial_data.apply(determine_area_type_from_row, axis=1)


In [20]:
industrial_data = industrial_data[columns_to_keep]

In [21]:
industrial_data

,geometry,name,id,description,area_type,category
0,POINT (4.72504 52.1378),NaN,32240050,NaN,Wind turbines,Energy/Industrial
1,POINT (4.63785 52.07556),NaN,304794992,NaN,High infrastructures,Energy/Industrial
2,POINT (4.69774 52.11005),Steektermolen,308551423,Poldermolen,Wind turbines,Energy/Industrial
3,POINT (4.67026 52.11973),De Eendracht,356719351,Korenmolen,Wind turbines,Energy/Industrial
4,POINT (4.59431 52.12524),De Rijnenburger,357384437,Poldermolen,Wind turbines,Energy/Industrial
...,...,...,...,...,...,...
279,"LINESTRING (4.63629 52.12767, 4.63621 52.12778)",NaN,1342601330,NaN,Power lines,Energy/Industrial
280,"LINESTRING (4.6361 52.12762, 4.63602 52.12773)",NaN,1342601331,NaN,Power lines,Energy/Industrial
281,"LINESTRING (4.63573 52.1278, 4.63571 52.12783)",NaN,1342601337,NaN,Power lines,Energy/Industrial
282,"LINESTRING (4.63571 52.1277, 4.63569 52.12774)",NaN,1342601338,NaN,Power lines,Energy/Industrial


In [22]:
industrial_data.to_file(f'/Users/cmartens/Documents/Thesis/model/OSM_data/output/{folder_name}/energy_industrial_{file_name}')

## Land use

In [23]:
land_use_tags = {
    "landuse": [
        "industrial",
        "commercial",
        "retail",
        "residential",
        "recreation_ground",
        "farmland",
        "orchard"
    ]
}

In [24]:
land_use_data = ox.features_from_place(boundaries, tags=land_use_tags)

In [25]:
land_use_data.reset_index(inplace=True)

In [26]:
land_use_data['category'] = "Land use"

In [27]:
land_use_data["area_type"] = land_use_data.apply(determine_area_type_from_row, axis=1)

In [28]:
for column in columns_to_keep:
    if column not in land_use_data.columns:
        land_use_data[column] = None

land_use_data = land_use_data[columns_to_keep]

In [29]:
land_use_data

,geometry,name,id,description,area_type,category
0,"POLYGON ((4.6013 52.04019, 4.60232 52.03956, 4...",NaN,1364714,None,Agricultural lands,Land use
1,"POLYGON ((4.63494 52.07313, 4.63499 52.07314, ...",NaN,8918862,None,Residential areas,Land use
2,"MULTIPOLYGON (((4.65211 52.03166, 4.65177 52.0...",Coenecoop,12819471,None,Industrial zones,Land use
3,"POLYGON ((4.57475 52.09095, 4.574 52.09099, 4....",NaN,15420092,None,Agricultural lands,Land use
4,"MULTIPOLYGON (((4.61761 52.127, 4.62166 52.126...",Zonnepark Alphen aan den Rijn - N11,17884208,None,Power plants,Land use
...,...,...,...,...,...,...
627,"POLYGON ((4.70239 52.09252, 4.70231 52.09246, ...",NaN,1387998613,None,Industrial zones,Land use
628,"POLYGON ((4.62439 52.10661, 4.62438 52.10664, ...",NaN,1388510476,None,Agricultural lands,Land use
629,"POLYGON ((4.54968 52.10665, 4.54969 52.10663, ...",NaN,1388752147,None,Industrial zones,Land use
630,"POLYGON ((4.54794 52.10648, 4.54857 52.10664, ...",NaN,1388752153,None,Residential areas,Land use


In [30]:
land_use_data.to_file(f'/Users/cmartens/Documents/Thesis/model/OSM_data/output/{folder_name}/land_use_{file_name}')

## Nature

In [31]:
nature_tags = {
    "landuse": ["forest", "wood", "scrub", "meadow", "grass", "grassland"],
    "natural": ["wetland"],
    "wetland": ["marsh", "bog", "fen"]
}

In [32]:
nature_data = ox.features_from_place(boundaries, tags=nature_tags)


In [33]:
nature_data.reset_index(inplace=True)

In [34]:
nature_data["area_type"] = nature_data.apply(determine_area_type_from_row, axis=1)

In [35]:
nature_data['category'] = "Nature"

In [36]:
for column in columns_to_keep:
    if column not in nature_data.columns:
        nature_data[column] = None

nature_data = nature_data[columns_to_keep]

In [37]:
nature_data

,geometry,name,id,description,area_type,category
0,"POLYGON ((4.69251 52.07553, 4.69254 52.07557, ...",NaN,1363235,NaN,Meadows and open grass,Nature
1,"POLYGON ((4.6739 52.04866, 4.67385 52.04865, 4...",NaN,1363257,NaN,Meadows and open grass,Nature
2,"POLYGON ((4.68556 52.07376, 4.68561 52.07386, ...",NaN,1363274,NaN,Meadows and open grass,Nature
3,"POLYGON ((4.67879 52.06182, 4.67876 52.06175, ...",NaN,1363276,NaN,Meadows and open grass,Nature
4,"POLYGON ((4.73308 52.13087, 4.73301 52.13091, ...",NaN,1364044,NaN,Meadows and open grass,Nature
...,...,...,...,...,...,...
8283,"POLYGON ((4.62834 52.15712, 4.62858 52.15787, ...",NaN,1390446859,NaN,Meadows and open grass,Nature
8284,"POLYGON ((4.62832 52.15712, 4.62831 52.1571, 4...",NaN,1390446860,NaN,Meadows and open grass,Nature
8285,"POLYGON ((4.69037 52.08018, 4.6903 52.08018, 4...",NaN,1390471105,NaN,Meadows and open grass,Nature
8286,"POLYGON ((4.69697 52.08049, 4.69472 52.08096, ...",NaN,1390471106,NaN,Meadows and open grass,Nature


In [38]:
nature_data.to_file(f'/Users/cmartens/Documents/Thesis/model/OSM_data/output/{folder_name}/nature_{file_name}')

## Water

In [39]:
water_tags = {
    "waterway": ["river", "stream", "canal"],
    "water": ["lake", "reservoir", "pond", "bog", "fen"] 
}

In [40]:
water_data = ox.features_from_place(boundaries, tags=water_tags)

In [41]:
water_data.reset_index(inplace=True)

In [42]:
water_data["area_type"] = water_data.apply(determine_area_type_from_row, axis=1)


In [43]:
water_data['category'] = "Water"

In [44]:
for column in columns_to_keep:
    if column not in water_data.columns:
        water_data[column] = None

water_data = water_data[columns_to_keep]

In [45]:
water_data

,geometry,name,id,description,area_type,category
0,"LINESTRING (4.73915 52.12086, 4.73907 52.12087...",NaN,32761837,None,"Rivers, canals and streams",Water
1,"LINESTRING (4.73959 52.12132, 4.74007 52.12124)",NaN,33269068,None,"Rivers, canals and streams",Water
2,"LINESTRING (4.6322 52.08696, 4.63227 52.08697,...",NaN,54513879,None,"Rivers, canals and streams",Water
3,"LINESTRING (4.63535 52.08671, 4.63548 52.0867,...",NaN,54515031,None,"Rivers, canals and streams",Water
4,"LINESTRING (4.64627 52.08619, 4.64627 52.08635)",NaN,54516132,None,"Rivers, canals and streams",Water
...,...,...,...,...,...,...
298,"LINESTRING (4.53899 52.10469, 4.53909 52.10469)",Nieuwe Wetering,1388616653,None,"Rivers, canals and streams",Water
299,"LINESTRING (4.53796 52.1048, 4.53805 52.1048)",Nieuwe Wetering,1388616654,None,"Rivers, canals and streams",Water
300,"LINESTRING (4.5372 52.10485, 4.53796 52.1048)",Nieuwe Wetering,1388616655,None,"Rivers, canals and streams",Water
301,"LINESTRING (4.53709 52.10485, 4.5372 52.10485)",Nieuwe Wetering,1388616656,None,"Rivers, canals and streams",Water


In [46]:
water_data.to_file(f'/Users/cmartens/Documents/Thesis/model/OSM_data/output/{folder_name}/water_{file_name}')

## Public facility

In [47]:
public_facility_tags = {
    "amenity": [
        "school",
        "kindergarten",
        "university",
        "hospital",
        "nursing_home",
        "prison",
        "place_of_worship"],
    "historic": ["*"], 
    "tourism": ["attraction"],
    "landuse": ["cemetery"],
    "leisure": ["park"]
}

In [48]:
public_data = ox.features_from_place(boundaries, tags=public_facility_tags)

In [49]:
public_data.reset_index(inplace=True)

In [50]:
public_data['category'] = 'Public facility'

In [51]:
public_data["area_type"] = public_data.apply(determine_area_type_from_row, axis=1)

In [52]:
for column in columns_to_keep:
    if column not in public_data.columns:
        public_data[column] = None

public_data = public_data[columns_to_keep]

In [53]:
public_data

,geometry,name,id,description,area_type,category
0,POINT (4.67317 52.1453),Kerkelijk Centrum De Bron,588393739,NaN,Religious sites,Public facility
1,POINT (4.55353 52.07647),School met de Bijbel,595699060,NaN,Schools and universities,Public facility
2,POINT (4.54713 52.07625),Arnoldus van Os,595699063,NaN,Schools and universities,Public facility
3,POINT (4.65736 52.03629),Koninkrijkszaal van Jehova's Getuigen,602522720,NaN,Religious sites,Public facility
4,POINT (4.66191 52.07526),Vergadering van Gelovigen,602522728,NaN,Religious sites,Public facility
...,...,...,...,...,...,...
234,"POLYGON ((4.67567 52.11966, 4.67586 52.11992, ...",Voormalige Begraafplaats Lage Zijde (Geruimd),1339327572,NaN,Cultural sites,Public facility
235,"POLYGON ((4.65086 52.11279, 4.65086 52.11291, ...",NaN,1355101282,NaN,Cultural sites,Public facility
236,"POLYGON ((4.64993 52.11265, 4.64994 52.11277, ...",NaN,1355101283,NaN,Cultural sites,Public facility
237,"POLYGON ((4.64157 52.11691, 4.64156 52.11691, ...",NaN,1355110261,NaN,Cultural sites,Public facility


In [54]:
public_data.to_file(f'/Users/cmartens/Documents/Thesis/model/OSM_data/output/{folder_name}/public_facilities_{file_name}')

## Airspace/Aviation

In [55]:
aviation_tags = {
    "aeroway": [
        "aerodrome",       # Airports and airfields
        "airstrip",        # Basic landing strips
        "heliport",        # Dedicated helicopter facilities
        "helipad",         # Individual helicopter landing pads
        "runway",          # Aircraft runways
        "taxiway",         # Taxi paths for aircraft
        "apron",           # Aircraft parking areas
        "terminal",        # Passenger terminals
        "gate",            # Boarding gates
        "hangar",          # Aircraft storage buildings
        "control_tower",   # Air traffic control towers
        "navigationaid",   # Navigation aids like beacons
        "launchpad",       # Rocket launch facilities
        "spaceport",       # Spaceflight launch sites
        "highway_strip"    # Roadways used as emergency runways
    ],
    "man_made": [
        "control_tower"    # Air traffic control towers
    ]
}


In [56]:
aviation_data = ox.features_from_place(boundaries, tags=aviation_tags)

InsufficientResponseError: No matching features. Check query location, tags, and log.

In [57]:
aviation_data.reset_index(inplace=True)

NameError: name 'aviation_data' is not defined

In [58]:
aviation_data

NameError: name 'aviation_data' is not defined

In [59]:
aviation_data["area_type"] = aviation_data.apply(determine_area_type_from_row, axis=1)

NameError: name 'aviation_data' is not defined

In [60]:
aviation_data

NameError: name 'aviation_data' is not defined

In [61]:
aviation_data['category'] = "Airports/Aviation"

NameError: name 'aviation_data' is not defined

In [62]:
for column in columns_to_keep:
    if column not in aviation_data.columns:
        aviation_data[column] = None

aviation_data = aviation_data[columns_to_keep]


NameError: name 'aviation_data' is not defined

In [63]:
aviation_data

NameError: name 'aviation_data' is not defined

In [64]:
aviation_data.to_file(f'/Users/cmartens/Documents/Thesis/model/OSM_data/output/{folder_name}/aviation_{file_name}')

NameError: name 'aviation_data' is not defined

# Postnl

In [65]:
post_tags = {"amenity": ["post_office", "post_box", "post_depot"]}

In [66]:
post_data = ox.features_from_place(boundaries, tags=post_tags)

In [67]:
post_data.reset_index(inplace=True)

In [68]:
for column in columns_to_keep:
    if column not in post_data.columns:
        post_data[column] = None

post_data = post_data[columns_to_keep]

In [69]:
post_data['area_type'] = 'postnl point'
post_data['category'] = 'PostNL'

In [70]:
post_data

,geometry,name,id,description,area_type,category
0,POINT (4.64633 52.04288),NaN,302330426,None,postnl point,PostNL
1,POINT (4.63595 52.03715),PostNL,302330621,None,postnl point,PostNL
2,POINT (4.66225 52.04385),NaN,303648200,None,postnl point,PostNL
3,POINT (4.65306 52.03063),NaN,311524912,None,postnl point,PostNL
4,POINT (4.6482 52.06488),PostNL Pakketpunt,348127872,None,postnl point,PostNL
...,...,...,...,...,...,...
76,POINT (4.65087 52.12642),NaN,7105101668,None,postnl point,PostNL
77,POINT (4.64873 52.13501),NaN,7220656837,None,postnl point,PostNL
78,POINT (4.65486 52.0234),NaN,8901678928,None,postnl point,PostNL
79,POINT (4.5786 52.09552),NaN,11333642753,None,postnl point,PostNL


In [71]:
post_data.to_file(f'/Users/cmartens/Documents/Thesis/model/OSM_data/output/{folder_name}/post_{file_name}')

# Concat

In [72]:
categories = [
    transportation_data,
    industrial_data,
    land_use_data,
    nature_data,
    water_data,
    public_data,
    #aviation_data,
    post_data
]

# Filter out None
categories = [cat for cat in categories if cat is not None]

# Concat if there's anything to concat
if categories:
    df = pd.concat(categories, ignore_index=True)
else:
    print("No data to combine.")


In [73]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df['area_type'].value_counts())

area_type
Pedestrian and cycling paths      10106
Tracks and rural access roads      6576
Meadows and open grass             6384
Regional roads                     4069
Forests and woodlands              1853
Living and residential streets      930
Motorways and major roads           728
Agricultural lands                  456
Rivers, canals and streams          291
Communication towers                208
Railways                            152
Residential areas                    98
Schools and universities             97
postnl point                         81
Cultural sites                       73
Religious sites                      63
Industrial zones                     54
Wetlands                             51
Power lines                          40
Wind turbines                        20
Bridges                              17
High infrastructures                 13
Retail zones                         13
Lakes and ponds                      12
Recreational zones            

In [74]:
gdf = gpd.GeoDataFrame(df, crs="EPSG:4326")

In [75]:
gdf.to_file(f'/Users/cmartens/Documents/Thesis/model/OSM_data/output/{folder_name}/all_{file_name}', driver='GeoJSON')